In [21]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split  
from math import radians, cos, sin, asin, sqrt
import math




def haversine(row):   
    lon1, lat1, lon2, lat2 = map(radians, [row[0], row[1], row[2], row[3]])  
 
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 
    return c * r * 1000

In [22]:
data = pd.read_csv("data/final_2g_tr.csv")
test_data = pd.read_csv("data/final_2g_te.csv")
public_data = pd.read_csv("data/final_2g_gongcan.csv")

column = [2,3,13,14,15,16,17,18,23,10,11]

train_select_column = data.iloc[:, column]
test_select_column = test_data.iloc[:,column]
all_select_column = train_select_column.append(test_select_column)
useful_public_data = public_data.iloc[:, [4,5,6,13,14]]
print train_select_column

print useful_public_data

             IMSI  SRNCID  CellID_1  EcNo_1  RSCP_1  RTT_1  UE_Rx_Tx_1  \
0      1234567890    6290     49714      -5     -62   -999        -999   
1      1234567890    6290     49714      -5     -62   -999        -999   
2      1234567890    6290     49714      -5     -61   -999        -999   
3      1234567890    6290     49714      -5     -61   -999        -999   
4      1234567890    6290     49714      -5     -61   -999        -999   
5      1234567890    6290     49714      -5     -61   -999        -999   
6      1234567890    6290     49714      -5     -61   -999        -999   
7      1234567890    6290     49714      -5     -59   -999        -999   
8      1234567890    6290     49714      -5     -59   -999        -999   
9      1234567890    6290     49714      -5     -59   -999        -999   
10     1234567890    6290     49714      -5     -59   -999        -999   
11     1234567890    6290     49714      -5     -59   -999        -999   
12     1234567890    6290     49714   

In [23]:
def link(row):
    if math.isnan(row[1]) or math.isnan(row[2]):
        return "NULL"
    else:                           
        return str(int(row[1]))+"-"+str(int(row[2]))
                                

res = useful_public_data.apply(link, axis=1)

useful_public_data["key"] = res
useful_public_data.pop("CGI")
useful_public_data.pop("CI")
useful_public_data.pop("LAC")
useful_public_data.columns = ["Lon", "Lat", "key"]
print useful_public_data

              Lon        Lat         key
0      121.611955  31.255286  6339-41251
1      121.569053  31.209631  6314-17442
2      121.512601  31.239761  6159-52872
3      121.476434  31.155627  6218-61555
4      121.536511  31.197501  6314-17537
5      121.561921  31.165889  6348-64177
6             NaN        NaN  6322-58690
7      121.508871  31.617911   6341-4130
8      121.816981  31.509441   6357-9234
9      121.582861  31.325551  6308-58881
10     121.636008  31.196271  6334-37618
11     121.639701  31.240851  6334-37377
12     121.617591  31.272711  6340-33442
13            NaN        NaN  6338-62562
14     121.548054  31.208557  6316-62435
15     121.392467  31.301138  6255-42163
16     121.456241  30.919251  6296-54322
17     121.888361  31.545211   6357-8355
18     121.307711  31.153371  6210-26801
19     121.454731  31.236305  6299-62483
20     121.346911  30.799611  6172-62625
21     121.454731  31.236305  6299-62482
22     121.182941  30.931171  6171-58850
23     121.32381

c:\python27\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [24]:
result_column = [48]
train_result_column = data.iloc[:,result_column]
test_result_column = test_data.iloc[:,result_column]
all_result_column = train_result_column.append(test_result_column)


res = all_select_column.apply(link, axis=1)
all_select_column["key"] = res
print all_select_column

             IMSI  SRNCID  CellID_1  EcNo_1  RSCP_1  RTT_1  UE_Rx_Tx_1  \
0      1234567890    6290     49714      -5     -62   -999        -999   
1      1234567890    6290     49714      -5     -62   -999        -999   
2      1234567890    6290     49714      -5     -61   -999        -999   
3      1234567890    6290     49714      -5     -61   -999        -999   
4      1234567890    6290     49714      -5     -61   -999        -999   
5      1234567890    6290     49714      -5     -61   -999        -999   
6      1234567890    6290     49714      -5     -61   -999        -999   
7      1234567890    6290     49714      -5     -59   -999        -999   
8      1234567890    6290     49714      -5     -59   -999        -999   
9      1234567890    6290     49714      -5     -59   -999        -999   
10     1234567890    6290     49714      -5     -59   -999        -999   
11     1234567890    6290     49714      -5     -59   -999        -999   
12     1234567890    6290     49714   

In [28]:
res = pd.merge(all_select_column, useful_public_data, how="left", on="key")

res["FinalLon"] =  res["Longitude"] - res["Lon"]
res["FinalLat"] = res["Latitude"] - res["Lat"]


res.pop("Longitude")

res.pop("Latitude")
all_res = pd.DataFrame()
all_res["Lon"] = res["FinalLon"]
all_res["Lat"] = res["FinalLat"]

models = []
draw_point = []
for i in range(10):
    single_point = 0
    for name, group in res.groupby('key'):
    #     print name
    #     print group
        result = pd.DataFrame()
        result["Longitude"] = group["FinalLon"]
        result["Latitude"] = group["FinalLat"]
        group.pop("FinalLon")
        group.pop("FinalLat")
        group.pop("key")
        
        train_X, test_X, train_Y, test_Y = train_test_split(group, result, test_size=0.2)
        features = len(train_X.columns)
        print train_X.shape[0]
        print test_X.shape[0]
    
        clr = RandomForestRegressor(max_features = int(sqrt(features)))
                                
        clr.fit(train_X, train_Y) 
                            
        regressor_result = clr.predict(test_X)
        
        print clr.score(test_X, test_Y)
        x = [p[0] for p in regressor_result.tolist()]
        y = [p[1] for p in regressor_result.tolist()]
        test_Y.insert(2, "Pre_lon", x)
        test_Y.insert(3, "Pre_lat", y)
    #     print test_Y
    #     test_Y["Pre_lon"] = test_Y["Pre_lon"] + group["Lon"]
    #     test_Y["Pre_lat"] = test_Y["Pre_lat"] + group["Lat"]
        final_res = test_Y.apply(haversine, axis=1)
        print str(final_res.mean()) + "m"
        single_point += final_res.mean() * final_res.shape[0]
        models.append(clr)
    draw_point.append(single_point)

In [ ]:
for c in clr:
    regressor_result = c.predict(res)
    x = [p[0] for p in regressor_result.tolist()]
    y = [p[1] for p in regressor_result.tolist()]
    all_res.insert(2, "Pre_lon", x)
    all_res.insert(3, "Pre_lat", y)
    final_res = all_res.apply(haversine, axis=1)
    print str(final_res.mean()) + "m"

In [ ]:
print train_X

features = len(train_X.columns)

clr = RandomForestRegressor(max_features = int(sqrt(features)))
                            
clr.fit(train_X, train_Y) 
                            
regressor_result = clr.predict(test_X)
                            
print clr.score(test_X, test_Y)